In [ ]:
from GBP.data import DataGenerator
from GBP.gbp import run_GaBP_SYNC_ACCELERATED, run_GaBP_HARDWARE_ACCELERATED_RESIDUAL_T_nP, run_GaBP_HARDWARE_ACCELERATED_RESIDUAL_T_MAX, run_GaBP_HARDWARE_ACCELERATED, run_GaBP_HARDWARE_ACCELERATED_RESIDUAL, run_GaBP_HARDWARE_ACCELERATED_RESIDUAL_NEW, run_GaBP_HARDWARE_ACCELERATED_EXCLUSION_NEIGHBOURS_ACROSS_STREAMS, run_GaBP_HARDWARE_ACCELERATED_EXCLUSION_NEIGHBOURS_ACROSS_STREAMS_STOCHASTIC
# , run_GaBP_HARDWARE_ACCELERATED_EXCLUSION_NEIGHBOURS_ACROSS_2_STREAMS
from GBP.utilities import HiddenPrints
from GBP.visulisation import set_plot_options, get_plot_colors, NetworkxGraph, AnalyzeResult
 
import warnings
import matplotlib
import numpy as np
import math
import random

# Option 1: Suppress all warnings
warnings.filterwarnings("ignore")

set_plot_options()
colors = get_plot_colors()

data_gen = DataGenerator()
result_analyzer = AnalyzeResult()

# SETUP

In [ ]:
# Number of nodes
num_nodes = 1000
sync_convergence_threshold = 1*10**-8

# starting mae for convergence
starting_mae_for_simulation = 10

# RUN FIRST TIME
sum_of_iterations = 0
num_iterations = 1
for _ in range(1000):
    A,b = data_gen.get_1D_line_matrix_PSD_difficult(num_nodes, eigenvalue_spread=1e-5, regularization_strength=1e-2, noise_strength=1e-2)
    graph = NetworkxGraph(A)
    P_i, mu_i, N_i, P_ii, mu_ii, P_ij, mu_ij, iter_dist, stand_divs, means, iteration = run_GaBP_SYNC_ACCELERATED(A, b, max_iter=1000, mae=False, convergence_threshold=sync_convergence_threshold, show=True)
    sum_of_iterations += iteration
    final_mean = list(mu_i)
    final_std = P_i
    if iteration < 1000:
        break
    else:
        print("=========== RESTART ===========")

print(f"AVE. SYNC ITERATIONS = {sum_of_iterations/num_iterations}")

In [ ]:
# # Number of nodes
# num_nodes = 1000
# sync_convergence_threshold = 1*10**-8

# # starting mae for convergence
# starting_mae_for_simulation = 10

# # RUN FIRST TIME
# sum_of_iterations = 0
# num_iterations = 1
# for _ in range(1000):
#     A,b = data_gen.get_2D_lattice_matrix_PSD_difficult(int(math.sqrt(num_nodes)), int(math.sqrt(num_nodes)), eigenvalue_spread=1e-5, regularization_strength=1e-2, noise_strength=1e-2)
#     graph = NetworkxGraph(A)
#     P_i, mu_i, N_i, P_ii, mu_ii, P_ij, mu_ij, iter_dist, stand_divs, means, iteration = run_GaBP_SYNC_ACCELERATED(A, b, max_iter=1000, mae=False, convergence_threshold=sync_convergence_threshold, show=True)
#     sum_of_iterations += iteration
#     final_mean = list(mu_i)
#     final_std = P_i
#     if iteration < 1000:
#         break
#     else:
#         print("=========== RESTART ===========")

# print(f"AVE. SYNC ITERATIONS = {sum_of_iterations/num_iterations}")

### 

In [4]:
async_convergence_threshold = 1*10**-5
sum_of_iterations = 0
num_iterations = 1
NODE_UPDT_PE = 1
PEs = 1
for it in range(0,num_iterations):
    # print(f"-------------- ITERATION = {it+1} --------------")
    P_i, mu_i, iteration, STARTING_MAE = run_GaBP_HARDWARE_ACCELERATED(A, b, caching=False, node_updates_per_pe=NODE_UPDT_PE, number_pes=PEs, TRUE_MEAN=final_mean, max_iter=1, mae=True, convergence_threshold=async_convergence_threshold, show=True)
    sum_of_iterations += iteration
    print(f"-------------- ITERATION = {it+1} => Streams = {iteration} --------------")

print(f"AVE. ASYNC ITERATIONS = {sum_of_iterations/num_iterations}")
print(STARTING_MAE)

KeyboardInterrupt: 

In [ ]:
# Number of nodes
num_nodes = 1000
sync_convergence_threshold = 1*10**-8

# starting mae for convergence
starting_mae_for_simulation = 10

# RUN FIRST TIME
sum_of_iterations = 0
num_iterations = 1
for _ in range(1000):
    b = starting_mae_for_simulation * b / STARTING_MAE
    graph = NetworkxGraph(A)
    P_i, mu_i, N_i, P_ii, mu_ii, P_ij, mu_ij, iter_dist, stand_divs, means, iteration = run_GaBP_SYNC_ACCELERATED(A, b, max_iter=200, mae=False, convergence_threshold=sync_convergence_threshold, show=False)
    sum_of_iterations += iteration
    final_mean = list(mu_i)
    final_std = P_i
    if iteration < 100000:
        break
    else:
        print("=========== RESTART ===========")

print(f"AVE. SYNC ITERATIONS = {sum_of_iterations/num_iterations}")

### START SIMULATION

In [ ]:
async_convergence_threshold = 1*10**-5

sum_of_iterations = 0
num_iterations = 3


NODE_UPDT_PE = 97
PEs = 3

for it in range(0,num_iterations):
    # print(f"-------------- ITERATION = {it+1} --------------")
    iteration = 100
    while iteration > 100-1:
        P_i, mu_i, iteration, _ = run_GaBP_HARDWARE_ACCELERATED(A, b, caching=False, node_updates_per_pe=NODE_UPDT_PE, number_pes=PEs, TRUE_MEAN=final_mean, max_iter=100, mae=True, convergence_threshold=async_convergence_threshold, show=True)
    sum_of_iterations += iteration
    print(f"-------------- ITERATION = {it+1} => Streams = {iteration} --------------")

print(f"AVE. ASYNC ITERATIONS = {sum_of_iterations/num_iterations}")

In [ ]:
async_convergence_threshold = 1*10**-5

sum_of_iterations = 0
num_iterations = 1


NODE_UPDT_PE = 137
PEs = 4

for it in range(0,num_iterations):
    # print(f"-------------- ITERATION = {it+1} --------------")
    iteration = 250
    while iteration > 250-1:
        P_i, mu_i, iteration, _, _ = run_GaBP_HARDWARE_ACCELERATED_RESIDUAL_T_MAX(A, b, ratio=1.0, caching=True, node_updates_per_pe=NODE_UPDT_PE, number_pes=PEs, TRUE_MEAN=final_mean, max_iter=250, mae=True, convergence_threshold=async_convergence_threshold, show=True)
        break
    sum_of_iterations += iteration
    print(f"-------------- ITERATION = {it+1} => Streams = {iteration} --------------")

print(f"AVE. ASYNC ITERATIONS = {sum_of_iterations/num_iterations}")

In [ ]:
async_convergence_threshold = 1*10**-5

sum_of_iterations = 0
num_iterations = 3


NODE_UPDT_PE = 97
PEs = 2

for it in range(0,num_iterations):
    # print(f"-------------- ITERATION = {it+1} --------------")
    iteration = 250
    while iteration > 250-1:
        P_i, mu_i, iteration = run_GaBP_HARDWARE_ACCELERATED_RESIDUAL(A, b, caching=True, node_updates_per_pe=NODE_UPDT_PE, number_pes=PEs, TRUE_MEAN=final_mean, max_iter=250, mae=True, convergence_threshold=async_convergence_threshold, show=True)
    sum_of_iterations += iteration
    print(f"-------------- ITERATION = {it+1} => Streams = {iteration} --------------")

print(f"AVE. ASYNC ITERATIONS = {sum_of_iterations/num_iterations}")

In [ ]:
async_convergence_threshold = 1*10**-5

sum_of_iterations = 0
num_iterations = 1


NODE_UPDT_PE = 97
PEs = 2

for it in range(0,num_iterations):
    # print(f"-------------- ITERATION = {it+1} --------------")
    iteration = 250
    while iteration > 250-1:
        P_i, mu_i, iteration = run_GaBP_HARDWARE_ACCELERATED_RESIDUAL_NEW(A, b, caching=True, node_updates_per_pe=NODE_UPDT_PE, number_pes=PEs, TRUE_MEAN=final_mean, max_iter=250, mae=True, convergence_threshold=async_convergence_threshold, show=True)
        break
    sum_of_iterations += iteration
    print(f"-------------- ITERATION = {it+1} => Streams = {iteration} --------------")

print(f"AVE. ASYNC ITERATIONS = {sum_of_iterations/num_iterations}")

In [ ]:
async_convergence_threshold = 1*10**-5

sum_of_iterations = 0
num_iterations = 1


NODE_UPDT_PE = 84
PEs = 4

for it in range(0,num_iterations):
    # print(f"-------------- ITERATION = {it+1} --------------")
    iteration = 100
    while iteration > 100-1:
        P_i, mu_i, iteration, T_max_lst, PADDING_ANALYTICS = run_GaBP_HARDWARE_ACCELERATED_RESIDUAL_T_nP(A, b, caching=True, node_updates_per_pe=NODE_UPDT_PE, number_pes=PEs, TRUE_MEAN=final_mean, max_iter=100, mae=True, convergence_threshold=async_convergence_threshold, show=True)
        break
    sum_of_iterations += iteration
    print(f"-------------- ITERATION = {it+1} => Streams = {iteration} --------------")

print(f"AVE. ASYNC ITERATIONS = {sum_of_iterations/num_iterations}")

In [ ]:
PADDING_ANALYTICS